In [4]:
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.common.by import By
from time import sleep
from datetime import datetime
import pytz
import json

In [ ]:
from driver import Bot
from colorama import Fore, Style
from selenium.webdriver.common.keys import Keys 

# Initialize the bot
bot = Bot()

# Test login and chat counting functionality
print(Fore.GREEN + "Starting WhatsApp Web test..." + Style.RESET_ALL)
# bot.login_and_count_chats()

# Keep the session open for manual testing
# When done testing, you can close the driver with:
# bot.quit_driver()

In [ ]:
bot.login()

In [32]:
# PEGAR TEXTO DA MENSAGEM
def get_all_messages():
    # Localize o contêiner da conversa
    conversation_container = bot.driver.find_element(
        By.CSS_SELECTOR, "div[data-tab='8']")
    
    messages = []
    previous_height = 0
    previous_message_count = 0

    while True:
        # Extraia mensagens atuais
        current_messages = conversation_container.find_elements(
            By.CSS_SELECTOR, ".message-in, .message-out")
        
        # Se não houver novas mensagens após o scroll, sair do loop
        if len(current_messages) == previous_message_count:
            break

        previous_message_count = len(current_messages)

        for message in current_messages[len(messages):]:
            try:
                copyable_text = message.find_elements(
                    By.CSS_SELECTOR, ".copyable-text")
                time, date, sender, utc_dt = None, None, None, None
                
                # Verificar se existe mensagem citada
                quoted_message = None
                quoted_elements = message.find_elements(
                    By.CSS_SELECTOR, "div[role='button'][aria-label='Quoted message']")
                if len(quoted_elements) > 0:
                    try:
                        quoted_sender = quoted_elements[0].find_elements(
                            By.CSS_SELECTOR, "span[dir='auto']._ao3e")
                        quoted_text = quoted_elements[0].find_elements(
                            By.CSS_SELECTOR, "span.quoted-mention._ao3e")
                        if len(quoted_text) > 0:
                            quoted_message = {
                                "sender": quoted_sender[0].text if len(quoted_sender) > 0 else '',
                                "text": quoted_text[0].text
                            }
                    except Exception as e:
                        print(f"Erro ao processar mensagem citada: {e}")

                text = ''   
                if len(copyable_text) > 0:
                    main_text_element = message.find_elements(
                        By.CSS_SELECTOR, "span.selectable-text.copyable-text")
                    if len(main_text_element) > 0:  # Se encontrou algum elemento
                        try:
                            text = main_text_element[0].text.strip()  # Pega o texto do primeiro elemento
                        except Exception as e:
                            print(f"Erro ao processar mensagem main: {e}")
   
                    
                    date_text = copyable_text[0].get_attribute("data-pre-plain-text")
                    if date_text:
                        # Extrai a data do formato '[HH:MM, DD/MM/YYYY] Nome: '
                        date_parts = date_text.split('] ')[0].replace('[', '').split(', ')
                        time = date_parts[0]
                        date = date_parts[1]
                        sender = date_text.split('] ')[1].replace(': ', '')
                        
                        # Convertendo para UTC
                        local_tz = pytz.timezone('America/Sao_Paulo')  # Assumindo fuso horário de São Paulo
                        datetime_str = f"{date} {time}"
                        local_dt = datetime.strptime(datetime_str, "%d/%m/%Y %H:%M")
                        local_dt = local_tz.localize(local_dt)
                        utc_dt = local_dt.astimezone(pytz.UTC)
            
                # text = message.text.strip()
                # # Remove o timestamp do final do texto (padrão HH:MM)
                # if text:
                #     # Remove o timestamp do final do texto (padrão HH:MM)
                #     text = ' '.join(text.split('\n')[:-1]) if '\n' in text else text

                message_data = {
                    "text": text,
                    "time": time,
                    "date": date,
                    "sender": sender,
                    "quoted_message": quoted_message,
                    "timestamp_utc": utc_dt.isoformat() if utc_dt else None, # TODO modificar se salvar em db
                }
                print(message_data)

                messages.append(message_data)
            except Exception as e:
                print(f"Erro ao processar uma mensagem: {e}")
        # Verifique se o scroll parou
        # current_height = conversation_container.size['height']
        # if current_height == previous_height:
        #     break  # Nenhum novo conteúdo foi carregado

        # previous_height = current_height
        
        # TODO VER FRASE EM PORTUGUES
        older_messages_button = bot.driver.find_elements(
                By.XPATH, "//button[.//div[contains(text(), 'Click here to get older messages from your phone.')]]")
        if len(older_messages_button) > 0:
            older_messages_button[0].click()
            sleep(10)
            
        # Role para o topo
        bot.driver.execute_script(
            "arguments[0].scrollTop = 0;", conversation_container)
        sleep(2)  # Aguarde o carregamento de novas mensagens
        
    print(messages)
    return list(messages)


# Coleta de mensagens
# all_messages = get_all_messages(conversation_container)

# Exibir mensagens coletadas
# for msg in all_messages:
#     print(msg)

In [5]:
def scroll_chats_list():
    chat_list_element = bot.driver.find_element(By.ID, "pane-side")
    # current_scroll = bot.driver.execute_script(
    #     "return arguments[0].scrollTop", chat_list_element)
    bot.driver.execute_script(
        "arguments[0].scrollTop = arguments[0].scrollTop + 1000", chat_list_element)
    sleep(2)


In [ ]:
#bot.click_first_chat_and_scroll()
# GET ALL CONTACTS NAMES FROM CHATS LIST
# Localizar a lista de chats
all_chats = set()  # Usar set para evitar duplicatas
last_seen_name = None  # Nome do último item visível na lista
# chat_list_element = bot.driver.find_element(By.ID, "pane-side")

while True:
    # Encontrar os chats visíveis
    chat_items = bot.driver.find_elements(By.XPATH, "//div[@aria-label='Chat list']//div[@role='listitem']")
    new_last_seen_name = None
    
    # se todos os chats já foram coletados, sair do loop, checa todos os no
    already_collected_chats = []
    # para teste somente 1 chat
    chat_items = [chat_items[0]]
    for chat in chat_items:
        try:
            # Captura o nome do chat e outras informações
            name = chat.find_element(By.XPATH, ".//span[@dir='auto']").text
            
            # Adicionar informações únicas ao conjunto
            if name not in all_chats:
                all_chats.add(f"{name}")
                chat.click()
                sleep(2)
                # get all messages and save in a file
                messages = get_all_messages()
                with open(f"{name}.json", "w") as f:
                    json.dump(messages, f)
                chat.click()
                sleep(2)
            else:
                already_collected_chats.append(name) 
            
            # Atualiza o último nome visto no final da lista
            # if i == len(chat_items) - 1:
            #     new_last_seen_name = name
            #     chat.click()
            #     sleep(2)
        except Exception as e:
            print(f"Erro ao processar um chat: {e}")
    break
    if len(already_collected_chats) == len(chat_items):
        break
    
    scroll_chats_list()
    # Fazer scroll para o último item
    # if chat_items:
    #     current_scroll = bot.driver.execute_script(
    #         "return arguments[0].scrollTop", chat_list_element)
    #     bot.driver.execute_script(
    #         "arguments[0].scrollTop = arguments[0].scrollTop + 1000", chat_list_element)
    #     sleep(2)  # Esperar o carregamento de novos chats

    # Condição de parada: o último nome não mudou
    # if new_last_seen_name == last_seen_name:
    #     break
    # last_seen_name = new_last_seen_name

print(all_chats)
print(len(all_chats))

In [5]:
# SEARCH THE CONTACT
message_input = bot.driver.find_element(
    By.XPATH, "//div[contains(@class, 'x1hx0egp')][@role='textbox']")
message_input.click()
first_contact = next(iter(all_chats))
message_input.send_keys(first_contact)
message_input.send_keys(Keys.ENTER)
sleep(2)


In [7]:
# FIND CHAT ELEMENTS
chat_list_div = bot.driver.find_element(By.CLASS_NAME, "x1y332i5")
chat_elements = chat_list_div.find_elements(By.CLASS_NAME, "_ak72")
len(chat_elements)

2

In [8]:
# CHAT CLICK
chat_elements[0].click()

In [9]:
# SMALL SCROLL ON CHATS LIST
chat_list_element = bot.driver.find_element(By.ID, "pane-side")
current_scroll = bot.driver.execute_script("return arguments[0].scrollTop", chat_list_element)
bot.driver.execute_script("arguments[0].scrollTop = arguments[0].scrollTop + 300", chat_list_element)